In [61]:
#kernel충돌 에러 뜨면 
#아나콘드prompt에서 pip install ipykernel --upgrade실행해보기

In [62]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [8]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader,SubsetRandomSampler

In [9]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore')

In [13]:
# CFG = {
#     'IMG_SIZE':224,
#     'EPOCHS':10,
#     'LEARNING_RATE':0.001,
#     'BATCH_SIZE':8,
#     'SEED':41
# }
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':5,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':16,
    'SEED':41
}

In [14]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [15]:
# def seed_everything(seed):
#     random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = True

# seed_everything(CFG['SEED']) # Seed 고정

In [16]:
df = pd.read_csv('./train.csv')
df

,id,img_path,A,B,C,D,E,F,G,H,I,J
0,TRAIN_00000,./train/TRAIN_00000.jpg,1,0,0,0,0,0,0,0,0,0
1,TRAIN_00001,./train/TRAIN_00001.jpg,1,0,0,0,0,0,0,0,0,0
2,TRAIN_00002,./train/TRAIN_00002.jpg,1,0,0,0,0,0,0,0,0,0
3,TRAIN_00003,./train/TRAIN_00003.jpg,1,0,0,0,0,0,0,0,0,0
4,TRAIN_00004,./train/TRAIN_00004.jpg,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
32989,TRAIN_32989,./train/TRAIN_32989.jpg,1,0,0,0,1,0,1,1,1,1
32990,TRAIN_32990,./train/TRAIN_32990.jpg,1,0,0,0,0,1,1,1,1,1
32991,TRAIN_32991,./train/TRAIN_32991.jpg,1,0,0,0,0,1,1,1,1,1
32992,TRAIN_32992,./train/TRAIN_32992.jpg,1,0,0,0,0,1,1,1,1,1


In [17]:
def get_labels(df):
    return df.iloc[:,2:].values

In [18]:
from sklearn.model_selection import train_test_split

#kfold를 하면, train-test를 나누어버리기 때문에, 따로 평가할 데이터를 빼놔야함
df, test, df_labels, test_labels = train_test_split(df, get_labels(df), test_size=0.1, random_state=42)

In [19]:
df.shape, df_labels.shape,test.shape,test_labels.shape

((29694, 12), (29694, 10), (3300, 12), (3300, 10))

In [20]:
#val, test, val_labels, test_labels = train_test_split(test, get_labels(test), test_size=0.5, random_state=42)

In [21]:
#val.shape,val_labels.shape,test.shape,test_labels.shape

In [22]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
            
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            
            
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [23]:
# train_transform = A.Compose([
#                             A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
#                             A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
#                             ToTensorV2()
#                             ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [24]:
# train_dataset = CustomDataset(train['img_path'].values, train_labels, train_transform)
# train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

# val_dataset = CustomDataset(val['img_path'].values, val_labels, test_transform)
# val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

test_dataset = CustomDataset(test['img_path'].values, test_labels, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [77]:
class BaseModel(nn.Module):
    def __init__(self, m,num_classes=10):
        super(BaseModel, self).__init__()
        
        if m==3:
            self.backbone = models.efficientnet_b3(pretrained=True)
        elif m==4:
            self.backbone = models.efficientnet_b4(pretrained=True)
        elif m==5:
            self.backbone = models.efficientnet_b5(pretrained=True)
        elif m==6:
            self.backbone = models.efficientnet_b6(pretrained=True)
        elif m==7:
            self.backbone = models.efficientnet_b7(pretrained=True)
            
        self.dropout = torch.nn.Dropout(0.5)
    
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.dropout(x)
        x = F.sigmoid(self.classifier(x))
        return x

In [78]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    val_acc = []
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            probs = model(imgs)
            
            loss = criterion(probs, labels)
            
            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = probs > 0.5
            batch_acc = (labels == preds).mean()
            
            val_acc.append(batch_acc)
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_acc = np.mean(val_acc)
    
    return _val_loss, _val_acc

In [79]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_acc = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_acc = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_acc)
            
        if best_val_acc < _val_acc:
            best_val_acc = _val_acc
            best_model = model
    
    return best_model

In [80]:
splits=KFold(n_splits=5,shuffle=True,random_state=42)
#optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)
splits

KFold(n_splits=5, random_state=42, shuffle=True)

In [81]:
#전체 데이터의 90%데이터 짜리는 kfold를 적용하기 위한 작업
df_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

df_dataset = CustomDataset(df['img_path'].values, df_labels, df_transform)

In [82]:
#각 데이터셋에서 학습한 모델5가지를 저장함
# models_list = []

# for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(df_dataset)))):

#     print('Fold {}'.format(fold + 1))

#     train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
#     val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)
    
#     # Define data loaders for training and testing data in this fold
#     #test데이터 10% 빼놓고 남은 90%을 이용해서 훈련-loss계산하는 식으로 진행함
#     trainloader = torch.utils.data.DataLoader(
#                       df_dataset, 
#                       batch_size=CFG['BATCH_SIZE'], sampler=train_subsampler)
    
#     valloader = torch.utils.data.DataLoader(
#                       df_dataset,
#                       batch_size=CFG['BATCH_SIZE'], sampler=val_subsampler)
    
#     #각 fold한 데이터셋마다 다른 모델를 적용함
#     #train데이터에서는 dropout을 적용하므로 model.eval()은 안씀
#     model = BaseModel(fold+3)
    
#     #각 데이터셋마다 학습모델을 다르게 설정하므로 model()를 계속 갱신함
#     optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

#     #efficient net b3,b4,b5,b6,b7으로 학습한 모델을 models에 전부 저장해놓음
#     train_model= train(model,optimizer,trainloader,valloader,scheduler,device) 
#     models_list.append(train_model)

Fold 1


  0%|          | 0/1485 [00:00<?, ?it/s]

  0%|          | 0/372 [00:00<?, ?it/s]

Epoch [1], Train Loss : [0.28935] Val Loss : [0.09375] Val ACC : [0.96257]


  0%|          | 0/1485 [00:00<?, ?it/s]

...

  0%|          | 0/372 [00:00<?, ?it/s]

Epoch [4], Train Loss : [0.04751] Val Loss : [0.07064] Val ACC : [0.97876]


  0%|          | 0/1485 [00:00<?, ?it/s]

  0%|          | 0/372 [00:00<?, ?it/s]

Epoch [5], Train Loss : [0.04061] Val Loss : [0.04427] Val ACC : [0.98458]


In [83]:
#model = BaseModel()
#model.eval()
#optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

#infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)
# model = MnistModel()
# model.to(device)# gpu에 모델 할당

#optimizer = optim.RAdam(model.parameters(), lr=1e-3)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=0.001, last_epoch=-1)
# criterion = nn.MultiLabelSoftMarginLoss()
# infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

## 10%빼놓은 성능평가용 test데이터로 학습한 모델 평가하기

## Hard voting VS Soft voting

1.hard voting은 다 더해서 3이상이면 1 아니면 0 
2.soft vodting은 다 더해서 확률이 2.5이상이면 1 아니면 0

In [28]:
#hard voting을 적용하므로 0.5이상이면 1, 0.5미만이면 0으로 정의한다음
#절반이상이 1이면 1, 0이면 0으로 판단함

#soft voting을 적용하면, 각 확률값을 더해서 2.5이상이면 1, 2.5미만이면 0으로 정의할 예정
#즉 5개의 모델이 0.5값을 전부 가지면 해당분류는 1
#0.5,0.5,0.5,0.5,0.4를 가지면 해당 분류는 0
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs,labels in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            #확률값을 이용해서 softvoting할 예정이므로 확률값을 저장함
            probs  = probs.cpu().detach().numpy()
            preds = probs > 0.5
            preds = preds.astype(int)
            predictions += probs.tolist()
    return predictions

In [29]:
# preds_models = []
# preds = inference(model, test_loader, device)

# #각 모델별 예측분류결과가 나옴
# preds_models.append(preds)

In [30]:
result = inference(model,test_loader, device)

NameError: name 'model' is not defined

In [90]:
a=[]
for idx in range(len(result)):
  if (test_labels[idx] == result[idx]).all():
    a.append(1)
  else:
    a.append(0)
a = np.array(a)

accuracy = a.sum() / len(a)
print('Accuracy : ',accuracy)

Accuracy :  0.9387878787878788


In [94]:
torch.save(model, './MOMOMODEL')

## label이 없는 original test데이터로 모델 적용한 결과

In [4]:
import pandas as pd
test = pd.read_csv('./test.csv')

In [5]:
test

,id,img_path
0,TEST_00000,./test/TEST_00000.jpg
1,TEST_00001,./test/TEST_00001.jpg
2,TEST_00002,./test/TEST_00002.jpg
3,TEST_00003,./test/TEST_00003.jpg
4,TEST_00004,./test/TEST_00004.jpg
...,...,...
1455,TEST_01455,./test/TEST_01455.jpg
1456,TEST_01456,./test/TEST_01456.jpg
1457,TEST_01457,./test/TEST_01457.jpg
1458,TEST_01458,./test/TEST_01458.jpg


In [10]:
from rembg.bg import remove as remove_bg
class TetsDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        image = remove_bg(image) ##바뀐부분
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [25]:
test_dataset = TetsDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


In [122]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
           
            predictions += probs.tolist()
    return predictions

In [26]:
# test_preds = []

# for m in models_list:
#     predict = inference(m, test_loader,device)
#     test_preds.append(predict)
#     #print(predict)
# #각 모델별 예측분류결과가 나옴
# test_preds

In [137]:
submit = pd.read_csv('./sample_submission.csv')

In [138]:
submit.iloc[:,1:] = result
submit.head()

,id,A,B,C,D,E,F,G,H,I,J
0,TEST_00000,0,1,0,0,0,0,1,0,1,0
1,TEST_00001,0,0,0,0,0,1,0,0,0,0
2,TEST_00002,1,1,0,0,1,1,0,0,0,0
3,TEST_00003,1,1,0,0,0,1,0,0,1,0
4,TEST_00004,1,1,0,0,1,0,0,0,0,0


In [139]:
submit.to_csv('./baseline_submit.csv', index=False)